In [1]:
import random
def generate_array(n):
    return random.sample(range(n), n)

In [2]:
%time
a = generate_array(100)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.39 µs


In [3]:
def merge(left, right):
    result = []
    i, j = 0, 0
    while i < len(left) or j < len(right):
        if i < len(left) and j < len(right):
            if left[i] < right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1
        elif i >= len(left):
            result.append(right[j])
            j += 1
        else:
            result.append(left[i])
            i += 1

    return result

def merge_sort(arr):
    assert isinstance(arr, list)
    if len(arr) <= 1:
        return arr
    middle = len(arr) // 2
    left, right = arr[:middle], arr[middle:]
    return merge(merge_sort(left), merge_sort(right))

In [6]:
from multiprocessing import Process, Queue, Pipe

def paralel_merge_sort(arr, fork_limit=8, q=None):
    assert isinstance(arr, list)
    if len(arr) <= 1:
        result = arr
    else:
        middle = len(arr) // 2
        left, right = arr[:middle], arr[middle:]
        if not fork_limit or len(arr) < 1000:
            result = merge(merge_sort(left), merge_sort(right))
        else:
            parent_conn, child_conn = Pipe()
            process = Process(target=paralel_merge_sort, args=(left,), 
                              kwargs={'fork_limit': fork_limit // 2, 'q': child_conn})
            process.start()
            right = paralel_merge_sort(right, fork_limit=fork_limit // 2)
            process.join()
            result = merge(parent_conn.recv(), right)

    if q:
        q.send(result)
        q.close()
    else:
        return result

In [80]:
n = 1000000
a = generate_array(n)
good = sorted(a)

In [81]:
%%time
sorted(a.copy()) == good

CPU times: user 404 ms, sys: 7.48 ms, total: 411 ms
Wall time: 409 ms


True

In [82]:
%%time
merge_sort(a.copy()) == good

CPU times: user 6.74 s, sys: 0 ns, total: 6.74 s
Wall time: 6.74 s


True

In [ ]:
%%time
paralel_merge_sort(a.copy()) == good